## 1. YOLO 모델 불러오기 및 예측 함수
- 이 코드는 Ultralytics YOLO 모델을 불러오고, 이미지에 대한 예측 결과를 Label Studio 형식으로 변환해 반환하는 함수(predict_yolo)를 정의합니다.

In [1]:
from ultralytics import YOLO

#MODEL_NAME = "yolov8n.pt"
#MODEL_NAME = "yolov8s.pt"
#MODEL_NAME = "yolov8m.pt"
MODEL_NAME = "./runs/detect/yolov8n_scratch_with_rotation_aug/weights/best.pt"
model = YOLO(MODEL_NAME)

def predict_yolo(images):
    results = model(images)
    predictions = []
    for result in results:
        #img_width, img_height = result.orig_shape
        img_height, img_width = result.orig_shape
        boxes = result.boxes.cpu().numpy()
        prediction = {'result': [], 'score': 0.0, 'model_version': MODEL_NAME}
        scores = []
        for box, class_id, score in zip(boxes.xywh, boxes.cls, boxes.conf):
            x, y, w, h = box
            prediction['result'].append({
                'from_name': 'label',
                'to_name': 'img',
                'original_width': int(img_width),
                'original_height': int(img_height),
                'image_rotation': 0,
                'value': {
                    'rotation': 0,
                    'rectanglelabels': [result.names[class_id]],
                    'width': w / img_width * 100,
                    'height': h / img_height * 100,
                    'x': (x - 0.5 * w) / img_width * 100,
                    'y': (y - 0.5 * h) / img_height * 100
                },
                'score': float(score),
                'type': 'rectanglelabels',
            })
            scores.append(float(score))
        prediction['score'] = min(scores) if scores else 0.0
        predictions.append(prediction)
    return predictions

## 2. YOLO 라벨 설정 생성
- YOLO 모델에서 제공하는 클래스 이름들을 Label Studio에 맞는 XML 형식의 라벨 설정으로 변환하여 출력하는 코드입니다.

In [2]:
yolo_labels = '\n'.join([f'<Label value="{label}"/>' for label in model.names.values()])
label_config = f'''
<View>
    <Image name="img" value="$image" zoom="true" width="100%" maxWidth="800" brightnessControl="true" contrastControl="true" gammaControl="true" />
    <RectangleLabels name="label" toName="img">
    {yolo_labels}
    </RectangleLabels>
</View>'''
print(label_config)


<View>
    <Image name="img" value="$image" zoom="true" width="100%" maxWidth="800" brightnessControl="true" contrastControl="true" gammaControl="true" />
    <RectangleLabels name="label" toName="img">
    <Label value="person"/>
<Label value="bicycle"/>
<Label value="car"/>
<Label value="motorcycle"/>
<Label value="airplane"/>
<Label value="bus"/>
<Label value="train"/>
<Label value="truck"/>
<Label value="boat"/>
<Label value="traffic light"/>
<Label value="fire hydrant"/>
<Label value="stop sign"/>
<Label value="parking meter"/>
<Label value="bench"/>
<Label value="bird"/>
<Label value="cat"/>
<Label value="dog"/>
<Label value="horse"/>
<Label value="sheep"/>
<Label value="cow"/>
<Label value="elephant"/>
<Label value="bear"/>
<Label value="zebra"/>
<Label value="giraffe"/>
<Label value="backpack"/>
<Label value="umbrella"/>
<Label value="handbag"/>
<Label value="tie"/>
<Label value="suitcase"/>
<Label value="frisbee"/>
<Label value="skis"/>
<Label value="snowboard"/>
<Label value

## 3. Label Studio 클라이언트 초기화
- .env 파일에서 API 키를 불러와 Label Studio SDK를 사용하기 위한 클라이언트를 초기화하는 코드입니다.

In [2]:
import os
from dotenv import load_dotenv
from label_studio_sdk.client import LabelStudio

load_dotenv()

API_KEY = os.getenv('LABEL_STUDIO_API_KEY')

client = LabelStudio(api_key=API_KEY)


## 4. 프로젝트 생성
- Label Studio에서 사용할 새 프로젝트를 생성하고, 간단한 설명과 함께 라벨 설정(label_config)을 적용합니다.

In [4]:
project = client.projects.create(
    title='Pre-annotation_test',
    description='Pre-annotation testing using the YOLO model',
    label_config=label_config
)

## 5. 모든 프로젝트 목록 확인
- 현재 로그인된 Label Studio 계정에서 사용 가능한 모든 프로젝트 목록을 확인합니다.

In [3]:
for project in client.projects.list():
    print(project.id, project.title)

33 yolo_polygon
32 tire_yolo_test
31 Pre-annotation_test
29 Object detection general picture direct import
28 Object detection general picture
27 Object detection modify
26 New Project #9
25 Object detection with yolov8n.pt
24 Object detection import json
21 Object detection
17 test2_json
16 json import
11 test_json
7 test_LocalStorage
1 test_import


## 6. 특정 프로젝트 가져오기
- 특정 ID(여기서는 31번)에 해당하는 프로젝트를 가져옵니다.

In [4]:
project = client.projects.get(id=33)

## 7. 로컬 Import 스토리지 생성
- 로컬 디렉터리에 있는 .json 파일들을 Label Studio에 가져오기 위한 Import 스토리지를 생성합니다.

In [ ]:
storage = client.import_storage.local.create(
    project=project.id,
    path='C:/Users/82102/dev/labelstudio-duckdb-dbt-test/data/tasks_generalpictures' , 
    regex_filter='.*\.json$',
    use_blob_urls=False,
    title='Json Files Storage',
    description='Import local json files'
)

print(f"Import storage created with ID: {storage.id}")

Import storage created with ID: 35


## 8. 로컬 Import 스토리지 동기화
- 앞서 생성한 Import 스토리지(storage.id)를 동기화하여 Label Studio에 태스크를 등록합니다.

In [8]:
client.import_storage.local.sync(id=storage.id)

LocalFilesImportStorage(id=35, type='localfiles', synchronizable=True, path='C:/Users/82102/dev/labelstudio-duckdb-dbt-test/data/tasks_generalpictures', regex_filter='.*\\.json$', use_blob_urls=False, last_sync=datetime.datetime(2025, 3, 13, 8, 49, 25, 326667, tzinfo=TzInfo(UTC)), last_sync_count=10, last_sync_job=None, status='completed', traceback=None, meta={'attempts': 1, 'time_queued': '2025-03-13 08:49:25.218165+00:00', 'time_in_progress': '2025-03-13 08:49:25.223752+00:00', 'time_last_ping': '2025-03-13 08:49:25.223752+00:00', 'time_completed': '2025-03-13 08:49:25.326667+00:00', 'duration': 0.102915, 'tasks_existed': 0}, title='Json Files Storage', description='Import local json files', created_at=datetime.datetime(2025, 3, 13, 8, 49, 22, 358450, tzinfo=TzInfo(UTC)), project=31)

## 9. 로컬 Export 스토리지 생성
- 주석(Annotation) 결과를 로컬 디렉터리에 JSON 파일로 내보내기 위한 Export 스토리지를 생성합니다.

In [ ]:

target_storage = client.export_storage.local.create(
    project=project.id,
    path='C:/Users/82102/dev/labelstudio-duckdb-dbt-test/data/export',
    title='Local Target Storage',
    description='Export annotations to local JSON files'
)

print(f"Target storage created with ID: {target_storage.id}")

Target storage created with ID: 36


## 10. 로컬 Export 스토리지 동기화
- 생성한 Export 스토리지(target_storage.id)를 동기화하여 주석 데이터를 로컬 디렉터리에 저장합니다.

In [10]:
client.export_storage.local.sync(id=target_storage.id)

LocalFilesExportStorage(id=36, type='localfiles', synchronizable=True, path='C:/Users/82102/dev/labelstudio-duckdb-dbt-test/data/export', regex_filter=None, use_blob_urls=False, last_sync=datetime.datetime(2025, 3, 13, 8, 49, 31, 243487, tzinfo=TzInfo(UTC)), last_sync_count=0, last_sync_job=None, status='completed', traceback=None, meta={'attempts': 2, 'time_queued': '2025-03-13 08:49:31.228836+00:00', 'time_in_progress': '2025-03-13 08:49:31.236365+00:00', 'time_last_ping': '2025-03-13 08:49:31.236365+00:00', 'time_completed': '2025-03-13 08:49:31.243487+00:00', 'duration': 0.007122, 'total_annotations': 0}, title='Local Target Storage', description='Export annotations to local JSON files', created_at=datetime.datetime(2025, 3, 13, 8, 49, 28, 292440, tzinfo=TzInfo(UTC)), can_delete_objects=None, project=31)

## 11. 모델 추론 및 예측 결과 생성
- Label Studio에 등록된 태스크 목록을 가져와, 각 이미지에 대해 YOLO 모델을 사용하여 예측을 수행하고, 예측 결과를 Label Studio에 저장합니다. 테스트를 위해 10개 정도까지만 예측을 수행합니다.

In [6]:

from PIL import Image
import requests
from tqdm import tqdm

tasks = [t.dict() for t in client.tasks.list(project=project.id)]

for i, task in enumerate(tqdm(tasks)):
    url = f'http://localhost:8080{task["data"]["image"]}'
    image = Image.open(requests.get(url, headers={'Authorization': f'Token {API_KEY}'}, stream=True).raw)
    predictions = predict_yolo([image])[0]
    # predictions 생성 시 task id는 원래 객체에 의존하므로, 만약 필요하면 t['id']로도 사용할 수 있음
    client.predictions.create(task=task["id"], result=predictions["result"], score=predictions["score"], model_version=predictions["model_version"])
    if i > 30:
        break

  0%|          | 0/25 [00:00<?, ?it/s]


0: 480x640 (no detections), 54.7ms
Speed: 2.6ms preprocess, 54.7ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)


  4%|▍         | 1/25 [00:02<00:55,  2.31s/it]


0: 480x640 (no detections), 42.8ms
Speed: 3.1ms preprocess, 42.8ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)


  8%|▊         | 2/25 [00:04<00:53,  2.31s/it]

...

 92%|█████████▏| 23/25 [00:53<00:04,  2.35s/it]


0: 480x640 (no detections), 44.2ms
Speed: 3.1ms preprocess, 44.2ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)


 96%|█████████▌| 24/25 [00:56<00:02,  2.34s/it]


0: 480x640 (no detections), 44.9ms
Speed: 4.1ms preprocess, 44.9ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)


100%|██████████| 25/25 [00:58<00:00,  2.34s/it]


## 12. 모델 버전 변경 함수
- YOLO 모델 버전을 변경하고, Label Studio 프로젝트 설정도 업데이트하는 함수를 정의합니다.

In [7]:
import requests
import json

def change_model_version(model_name):
    global MODEL_NAME  
    MODEL_NAME = model_name
    model = YOLO(MODEL_NAME)  
    

    headers = {
        'Authorization': f'Token {API_KEY}',
        'Content-Type': 'application/json'
    }
    
    update_data = {
        "model_version": MODEL_NAME,
        "show_collab_predictions": True
    }
    
    
    url = f"http://localhost:8080/api/projects/{project.id}/"
    response = requests.patch(url, headers=headers, data=json.dumps(update_data))
    
    if response.status_code == 200:
        print(f"모델을 {MODEL_NAME}으로 변경했습니다.")
        return True
    else:
        print(f"모델 변경 실패: {response.status_code}")
        print(response.text)
        return False


## 13. 모델 버전 변경 실행
- 앞서 정의한 change_model_version 함수를 사용하여 모델 버전을 변경합니다.
- 추론한 모델명으로 변경해야 UI의 Prediction score 에 반영이 됩니다.

In [8]:
change_model_version("yolov8s.pt")

모델을 yolov8s.pt으로 변경했습니다.


True

## 14. 태스크 객체 정보 확인
- Label Studio에서 가져온 태스크 객체(Task)의 속성과 메서드를 확인하고, 태스크 내용을 딕셔너리로 변환하여 키 목록을 출력합니다.

In [ ]:

tasks_list = list(tasks)
task = tasks_list[0]


print("Available attributes and methods:")
print(dir(task))


print("\nTask contents:")
print(task)


print("\nDictionary keys:")
print(task.keys())

Available attributes and methods:
['__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__ior__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__or__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__ror__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'clear', 'copy', 'fromkeys', 'get', 'items', 'keys', 'pop', 'popitem', 'setdefault', 'update', 'values']

Task contents:
{'id': 195, 'predictions': [], 'annotations': [], 'drafts': [], 'annotators': [], 'inner_id': 1, 'cancelled_annotations': 0, 'total_annotations': 0, 'total_predictions': 0, 'storage_filename': 'C:\\Users\\82102\\dev\\labelstudio-duckdb-dbt-test\\data\\tasks_generalpictures\\20230401_202954.json', 'draft_exists': False, 'updated_by': [], 'data': {'image': '/data/local-files/?d=data/samp

## 15. 태스크 소스파일(JSON 형식) 확인
- 태스크 데이터를 JSON 형태로 변환하여 구조를 확인합니다.


In [ ]:

import json
from datetime import datetime

tasks_list = list(tasks)
task = tasks_list[0]

def datetime_handler(obj):
    if isinstance(obj, datetime):
        return obj.isoformat()
    raise TypeError(f"Object of type {type(obj)} is not JSON serializable")


task_json = json.dumps(task, indent=2, default=datetime_handler)

print("Task Source Code (JSON format):")
print(task_json)

Task Source Code (JSON format):
{
  "id": 195,
  "predictions": [],
  "annotations": [],
  "drafts": [],
  "annotators": [],
  "inner_id": 1,
  "cancelled_annotations": 0,
  "total_annotations": 0,
  "total_predictions": 0,
  "storage_filename": "C:\\Users\\82102\\dev\\labelstudio-duckdb-dbt-test\\data\\tasks_generalpictures\\20230401_202954.json",
  "draft_exists": false,
  "updated_by": [],
  "data": {
    "image": "/data/local-files/?d=data/samples_generalpictures/20230401_202954.jpg"
  },
  "meta": {},
  "created_at": "2025-03-13T08:49:25.234320+00:00",
  "updated_at": "2025-03-13T08:49:25.234320+00:00",
  "is_labeled": false,
  "overlap": 1.0,
  "comment_count": 0,
  "unresolved_comment_count": 0,
  "project": 31,
  "comment_authors": [],
  "annotations_results": "",
  "predictions_results": "",
  "annotations_ids": "",
  "predictions_model_versions": "",
  "completed_at": null,
  "file_upload": null,
  "avg_lead_time": null,
  "last_comment_updated_at": null,
  "predictions_score